In [ ]:
import os
import time
#TOKEN_FILE = "/run/secrets/kubernetes.io/serviceaccount/token" # not working in RHODS 1.25
TOKEN_FILE = "/mnt/secret_token/token"
with open(TOKEN_FILE) as f:
  token = f.read().strip()
route = os.environ["DSP_ROUTE"]
pipeline_name = os.environ["PIPELINE_NAME"]
run_count = int(os.environ["RUN_COUNT"])
run_delay = int(os.environ["RUN_DELAY"])

In [ ]:
import os
import kfp
from pipeline import my_pipeline

In [ ]:
cert="/run/secrets/kubernetes.io/serviceaccount/ca.crt"
ex = None
for i in range(5):
  try:
    print(f'try #{i}')
    client = kfp.Client(host=route, existing_token=token, ssl_ca_cert=cert)
    print(f'try #{i} succeeded :)')
    break
  except Exception as e:
    import time;time.sleep(5)
    ex = e
else:
  raise Exception('Could not connect to Client the after multiple tries :/') from ex

In [ ]:
from kfp.compiler import Compiler
Compiler().compile(pipeline_func=my_pipeline, package_path="results/pipeline.yaml")
experiment = client.create_experiment(name=f"experiment-{pipeline_name}")
uploaded_pipeline = client.upload_pipeline(pipeline_package_path="results/pipeline.yaml", pipeline_name=pipeline_name)
versions = client.list_pipeline_versions(uploaded_pipeline.pipeline_id).pipeline_versions

In [ ]:
for i in range(run_count):
  run_name = f"{pipeline_name}-run{i}"
  client.run_pipeline(experiment.experiment_id, run_name, pipeline_id=uploaded_pipeline.pipeline_id, version_id=versions[0].pipeline_version_id, params={}, enable_caching=False)
  time.sleep(run_delay)